<a href="https://colab.research.google.com/github/min207/2023-1-ESAA/blob/main/%ED%83%84%EC%B2%9C_%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
#필요 패키지 로드
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
plt.rcParams['font.family'] = 'NanumGothic'

In [5]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##데이터 불러오기

###학습 데이터 불러오기

In [6]:
# 탄천 데이터 불러오기
tan = pd.read_csv('/content/drive/MyDrive/water/탄천수질데이터_전처리완료.csv', encoding='cp949')
tan.drop('Unnamed: 0', axis=1, inplace=True)

# 결측치 처리 방법1 - 결측치 행 삭제
tan1 = tan.dropna(axis=0)

# 결측치 처리 방법2 - 이전값으로 대체
from scipy import interpolate
tan2 = tan.interpolate(method='pad')
tan2 = tan2.interpolate(method='bfill')

##테스트 데이터 불러오기

###테스트 데이터 불러오기

In [7]:
# 탄천 데이터 불러오기
test2023 = pd.read_csv('/content/drive/MyDrive/water/탄천데이터2023_전처리완료.csv', encoding='cp949')
test2023.drop('Unnamed: 0', axis=1, inplace=True)

# 결측치 처리 방법1 - 결측치 행 삭제
test1 = test2023.dropna(axis=0)

# 결측치 처리 방법2 - 이전값으로 대체
from scipy import interpolate
test2 = test2023.interpolate(method='pad')
test2 = test2.interpolate(method='bfill')

##수질 등급 칼럼 생성

###1) 총질소 포함

In [8]:
# 함수 생성
def water_quality(a,b,c,d,e):
  if (a>=6.5) & (a<=8.5) & (b>=7.5) & (c<=2) & (d<=0.01) & (e<=2):
    return 1
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (c<=3) & (d<=0.02) & (e<=3):
    return 2
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (c<=4) & (d<=0.03) & (e<=4):
    return 3
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (c<=6) & (d<=0.05) & (e<=5):
    return 4
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (c<=10) & (d<=0.1) & (e<=8):
    return 5
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (c<=15) & (d<=0.15) & (e<=10):
    return 6
  else:
    return 7

# 함수 적용
tan2['수질등급'] = tan2.apply(lambda x: water_quality(x['pH'],x['용존산소(㎎/L)'],x['총질소(㎎/L)'],x['총인(㎎/L)'],x['총유기탄소']), axis=1)
test2['수질등급'] = test2.apply(lambda x: water_quality(x['pH'],x['용존산소(㎎/L)'],x['총질소(㎎/L)'],x['총인(㎎/L)'],x['총유기탄소']), axis=1)

In [9]:
# 분포 확인
print(tan2['수질등급'].value_counts())
print('\n')
print(test2['수질등급'].value_counts())

6    6434
5    6388
7    4197
4     501
Name: 수질등급, dtype: int64


5    2688
7    1456
6     942
4       2
Name: 수질등급, dtype: int64


###2) 총질소 제외

In [10]:
# 함수 생성
def water_quality2(a,b,d,e):
  if (a>=6.5) & (a<=8.5) & (b>=7.5) & (d<=0.01) & (e<=2):
    return 1
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.02) & (e<=3):
    return 2
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.03) & (e<=4):
    return 3
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.05) & (e<=5):
    return 4
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (d<=0.1) & (e<=8):
    return 5
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (d<=0.15) & (e<=10):
    return 6
  else:
    return 7

# 함수 적용
tan2['수질등급2'] = tan2.apply(lambda x: water_quality2(x['pH'],x['용존산소(㎎/L)'],x['총인(㎎/L)'],x['총유기탄소']), axis=1)
test2['수질등급2'] = test2.apply(lambda x: water_quality2(x['pH'],x['용존산소(㎎/L)'],x['총인(㎎/L)'],x['총유기탄소']), axis=1)

In [11]:
# 분포 확인
print(tan2['수질등급2'].value_counts())
print('\n')
print(test2['수질등급2'].value_counts())

5    6405
6    4790
7    4197
4    1946
3     182
Name: 수질등급2, dtype: int64


5    2337
7    1456
6     942
4     353
Name: 수질등급2, dtype: int64


###4) 25퍼센트 단위로 수질등급 생성 - 4등급

In [12]:
#pH 점수 구하는 함수
def get_pH(data):
  data['pH점수'] = np.where(abs(data['pH']-7.5)<0.25, 4,
                          np.where(abs(data['pH']-7.5)<0.5,3,
                                       np.where(abs(data['pH']-7.5)<0.75,2,1)))

In [13]:
#용존산소 점수 구하는 함수
def get_do(data):
  #용존산소 분위수 구하기
  yong1 = np.percentile(data['용존산소(㎎/L)'], 25)
  yong2 = np.percentile(data['용존산소(㎎/L)'], 50)
  yong3 = np.percentile(data['용존산소(㎎/L)'], 75)

  data['용존산소점수'] = np.where(data['용존산소(㎎/L)']<yong1, 1,
                          np.where(data['용존산소(㎎/L)']<yong2, 2,
                                       np.where(data['용존산소(㎎/L)']<yong3,3,4)))

In [14]:
#총인 점수 구하는 함수
def get_tp(data):
  #총인 분위수 구하기
  pho1 = np.percentile(data['총인(㎎/L)'], 25)
  pho2 = np.percentile(data['총인(㎎/L)'], 50)
  pho3 = np.percentile(data['총인(㎎/L)'], 75)

  data['총인점수'] = np.where(data['총인(㎎/L)']<pho1, 4,
                          np.where(data['총인(㎎/L)']<pho2, 3,
                                       np.where(data['총인(㎎/L)']<pho3,2,1)))

In [15]:
#총유기탄소 점수 구하는 함수
def get_toc(data):
  #총유기탄소 분위수 구하기
  carb1 = np.percentile(data['총유기탄소'], 25)
  carb2 = np.percentile(data['총유기탄소'], 50)
  carb3 = np.percentile(data['총유기탄소'], 75)

  data['총유기탄소점수'] = np.where(data['총유기탄소']<carb1, 4,
                          np.where(data['총유기탄소']<carb2, 3,
                                       np.where(data['총유기탄소']<carb3,2,1)))

In [193]:
#수질등급 구하는 함수
def get_grade25(data):
  data['수질등급'] = np.where((data['pH점수']+data['용존산소점수']+data['총인점수']+data['총유기탄소점수'])<5, 4,
                          np.where((data['pH점수']+data['용존산소점수']+data['총인점수']+data['총유기탄소점수'])<10,3,
                                       np.where((data['pH점수']+data['용존산소점수']+data['총인점수']+data['총유기탄소점수'])<15,2,1)))

In [194]:
#함수적용
get_pH(tan2)
get_do(tan2)
get_tp(tan2)
get_toc(tan2)
get_grade25(tan2)

In [195]:
tan2['수질등급'].value_counts()

2    8384
3    8234
1     825
4      77
Name: 수질등급, dtype: int64

#tan2 data

In [19]:
!pip3 install bayesian-optimization

##4개 등급 모델링

###train/test 데이터셋 생성하기

In [211]:
#함수적용 _ train
get_pH(tan2)
get_do(tan2)
get_tp(tan2)
get_toc(tan2)
get_grade25(tan2)

#함수적용 _ test
get_pH(test2)
get_do(test2)
get_tp(test2)
get_toc(test2)
get_grade25(test2)

###1-1. DT - 수온, 강우량 - 0.6284

In [ ]:
# train/test dataset 생성

train=tan2.loc[:,["날짜","수온","강우량","수질등급"]]
test=test2.loc[:,["날짜","수온","강우량","수질등급"]]

In [ ]:
X_train=train[['수온','강우량']]
y_train=train[['수질등급']]

X_test=test[['수온','강우량']]
y_test=test[['수질등급']]

In [ ]:
!pip3 install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def tree_bo1(max_depth, min_samples_split, min_samples_leaf, max_features):
  tree_params = {
      'max_depth' : int(round(max_depth)),
      'min_samples_split' : int(round(min_samples_split)),
      'min_samples_leaf' : int(round(min_samples_leaf)),
      'max_features' : int(round(max_features))
      }

  tree = DecisionTreeClassifier(**tree_params)
  tree.fit(X_train,y_train["수질등급"])

  score = accuracy_score(y_test["수질등급"], tree.predict(X_test))

  return score

In [ ]:
tree_parameter_bounds = {
                      'max_depth' : (1,10),
                      'min_samples_split' : (5,10),
                      'min_samples_leaf' : (5,15),
                      'max_features' : (2,20)
                      }

In [ ]:
BO_tree1 = BayesianOptimization(f = tree_bo1, pbounds = tree_parameter_bounds,random_state = 2023)

In [ ]:
BO_tree1.maximize(init_points = 50, n_iter = 5)

|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... |
-------------------------------------------------------------------------
| 1         | 0.7714    | 3.898     | 18.03     | 10.88     | 5.633     |
| 2         | 0.7634    | 2.272     | 10.42     | 5.221     | 8.636     |
| 3         | 0.7026    | 5.719     | 11.81     | 9.564     | 7.507     |
| 4         | 0.772     | 4.55      | 4.721     | 8.609     | 5.81      |
| 5         | 0.7714    | 4.042     | 5.246     | 8.91      | 5.178     |
| 6         | 0.7026    | 6.084     | 5.662     | 8.206     | 6.883     |
| 7         | 0.7714    | 2.656     | 3.871     | 9.549     | 5.979     |
| 8         | 0.7724    | 4.407     | 18.75     | 12.6      | 8.854     |
| 9         | 0.7026    | 6.37      | 16.25     | 13.1      | 9.903     |
| 10        | 0.7213    | 8.963     | 3.976     | 13.2      | 6.538     |
| 11        | 0.7714    | 3.353     | 9.303     | 10.53     | 8.128     |
| 12        | 0.7634    | 1.709     | 

In [ ]:
BO_tree1.max

{'target': 0.7724056603773585,
 'params': {'max_depth': 4.406728802978893,
  'max_features': 18.74957526706793,
  'min_samples_leaf': 12.60159713202054,
  'min_samples_split': 8.853821188293969}}

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=2023,max_depth=4,max_features=19,
                                min_samples_leaf=13,min_samples_split=9)
tree_clf.fit(X_train, y_train["수질등급"])
accuracy_train = accuracy_score(y_train["수질등급"], tree_clf.predict(X_train))
accuracy_test = accuracy_score(y_test["수질등급"], tree_clf.predict(X_test))
print('결정트리 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_train["수질등급"], tree_clf.predict(X_train)),'\n')
print('결정트리 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_test["수질등급"], tree_clf.predict(X_test)))

결정트리 학습 정확도: 0.6237
[[   0  308  517    0]
 [   0 3501 4883    0]
 [   0  807 7427    0]
 [   0    0   77    0]] 

결정트리 예측 정확도: 0.7724
[[   0   25    1    0]
 [   0 1276  893    0]
 [   0  204 2654    0]
 [   0    0   35    0]]


###2-1. RF- 수온, 강우량 - 0.6388

In [ ]:
# train/test dataset 생성

train=tan2.loc[:,["날짜","수온","강우량","수질등급"]]
test=test2.loc[:,["날짜","수온","강우량","수질등급"]]

In [ ]:
X_train=train[['수온','강우량']]
y_train=train[['수질등급']]

X_test=test[['수온','강우량']]
y_test=test[['수질등급']]

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def rf_bo1(max_depth, min_samples_split, min_samples_leaf, n_estimators):
  rf_params = {
      'max_depth' : int(round(max_depth)),
      'min_samples_split' : int(round(min_samples_split)),
      'min_samples_leaf' : int(round(min_samples_leaf)),
      'n_estimators' : int(round(n_estimators))
      }

  rf = RandomForestClassifier(**rf_params)
  rf.fit(X_train,y_train["수질등급"])

  score = accuracy_score(y_test["수질등급"], rf.predict(X_test))

  return score

In [ ]:
rf_parameter_bounds = {
                      'max_depth' : (1,10),
                      'min_samples_split' : (5,10),
                      'min_samples_leaf' : (5,15),
                      'n_estimators' : (50,200)
                      }

In [ ]:
BO_rf1 = BayesianOptimization(f = rf_bo1, pbounds = rf_parameter_bounds,random_state = 2023)

In [ ]:
BO_rf1.maximize(init_points = 50, n_iter = 5)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.7738    | 3.898     | 13.9      | 7.94      | 68.99     |
| 2         | 0.7628    | 2.272     | 9.679     | 5.11      | 159.1     |
| 3         | 0.7742    | 5.719     | 10.45     | 7.282     | 125.2     |
| 4         | 0.7763    | 4.55      | 6.512     | 6.804     | 74.31     |
| 5         | 0.7756    | 4.042     | 6.803     | 6.955     | 55.35     |
| 6         | 0.7742    | 6.084     | 7.035     | 6.603     | 106.5     |
| 7         | 0.7718    | 2.656     | 6.04      | 7.275     | 79.38     |
| 8         | 0.774     | 4.407     | 14.31     | 8.801     | 165.6     |
| 9         | 0.773     | 6.37      | 12.92     | 9.052     | 197.1     |
| 10        | 0.7335    | 8.963     | 6.098     | 9.099     | 96.14     |
| 11        | 0.7718    | 3.353     | 9.057     | 7.767     | 143.8     |
| 12        | 0.7636    | 1.709     | 

In [ ]:
BO_rf1.max

{'target': 0.7767295597484277,
 'params': {'max_depth': 4.635564437752466,
  'min_samples_leaf': 10.791859683184384,
  'min_samples_split': 9.589828696499449,
  'n_estimators': 77.70218803482284}}

In [ ]:
rf_clf = RandomForestClassifier(random_state=2023,max_depth=5,
                                min_samples_leaf=6,min_samples_split=8,
                                n_estimators=101)
rf_clf.fit(X_train, y_train["수질등급"])
accuracy_train = accuracy_score(y_train["수질등급"], rf_clf.predict(X_train))
accuracy_test = accuracy_score(y_test["수질등급"], rf_clf.predict(X_test))
print('랜덤 포레스트 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_train["수질등급"], rf_clf.predict(X_train)),'\n')
print('랜덤 포레스트 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_test["수질등급"], rf_clf.predict(X_test)))

랜덤 포레스트 학습 정확도: 0.6265
[[   0  376  449    0]
 [   0 3793 4591    0]
 [   0 1051 7183    0]
 [   0    0   77    0]] 

랜덤 포레스트 예측 정확도: 0.7765
[[   0   26    0    0]
 [   0 1372  797    0]
 [   0  279 2579    0]
 [   0    0   35    0]]


###3-1. GBM - 수온, 강우량 - 0.6333

In [ ]:
# train/test dataset 생성

train=tan2.loc[:,["날짜","수온","강우량","수질등급"]]
test=test2.loc[:,["날짜","수온","강우량","수질등급"]]

In [ ]:
X_train=train[['수온','강우량']]
y_train=train[['수질등급']]

X_test=test[['수온','강우량']]
y_test=test[['수질등급']]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def gbm_bo1(learning_rate, n_estimators):
  gbm_params = {
      'learning_rate' : int(round(learning_rate)),
      'n_estimators' : int(round(n_estimators))
      }

  gbm = GradientBoostingClassifier(**gbm_params)
  gbm.fit(X_train,y_train["수질등급"])

  score = accuracy_score(y_test["수질등급"], gbm.predict(X_test))

  return score

In [ ]:
gbm_parameter_bounds = {
                      'learning_rate' : (0.001,0.1),
                      'n_estimators' : (10,100)
                      }

In [ ]:
BO_gbm1 = BayesianOptimization(f = gbm_bo1, pbounds = gbm_parameter_bounds,random_state = 2023)

In [ ]:
BO_gbm1.maximize(init_points = 15, n_iter = 5)

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.4263    | 0.03288   | 90.14     |
| 2         | 0.4263    | 0.05922   | 21.39     |
| 3         | 0.4263    | 0.01499   | 52.11     |
| 4         | 0.4263    | 0.003187  | 75.45     |
| 5         | 0.4263    | 0.05291   | 59.04     |
| 6         | 0.4263    | 0.04618   | 55.12     |
| 7         | 0.4263    | 0.04005   | 23.61     |
| 8         | 0.4263    | 0.03673   | 24.59     |
| 9         | 0.4263    | 0.03446   | 26.23     |
| 10        | 0.4263    | 0.03971   | 13.21     |
| 11        | 0.4263    | 0.05692   | 28.31     |
| 12        | 0.4263    | 0.03274   | 43.89     |
| 13        | 0.4263    | 0.01922   | 19.36     |
| 14        | 0.4263    | 0.04604   | 27.63     |
| 15        | 0.4263    | 0.03847   | 93.75     |
| 16        | 0.4263    | 0.0775    | 68.07     |
| 17        | 0.4263    | 0.06465   | 100.0     |
| 18        | 0.4263    | 0.002829  | 99.99     |


In [ ]:
BO_gbm1.max

{'target': 0.42629716981132076,
 'params': {'learning_rate': 0.03287684209520481,
  'n_estimators': 90.13802066105328}}

In [ ]:
gbm_clf = GradientBoostingClassifier(random_state=2023,learning_rate=0.033,
                                n_estimators=90)
gbm_clf.fit(X_train, y_train["수질등급"])
accuracy_train = accuracy_score(y_train["수질등급"], gbm_clf.predict(X_train))
accuracy_test = accuracy_score(y_test["수질등급"], gbm_clf.predict(X_test))
print('GBM 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_train["수질등급"], gbm_clf.predict(X_train)),'\n')
print('GBM 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_test["수질등급"], gbm_clf.predict(X_test)))

GBM 학습 정확도: 0.6295
[[  73  377  375    0]
 [  38 3803 4543    0]
 [  25 1056 7153    0]
 [   0    0   77    0]] 

GBM 정확도: 0.7738
[[   0   26    0    0]
 [   4 1368  797    0]
 [  17  272 2569    0]
 [   0    0   35    0]]


###4-1.LGBM - 수온, 강우량 - 0.6207

In [ ]:
# train/test dataset 생성

train=tan2.loc[:,["날짜","수온","강우량","수질등급"]]
test=test2.loc[:,["날짜","수온","강우량","수질등급"]]

In [ ]:
X_train=train[['수온','강우량']]
y_train=train[['수질등급']]

X_test=test[['수온','강우량']]
y_test=test[['수질등급']]

In [ ]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def lgbm_bo1(learning_rate, n_estimators):
  lgbm_params = {
      'learning_rate' : int(round(learning_rate)),
      'n_estimators' : int(round(n_estimators))
      }

  lgbm = GradientBoostingClassifier(**lgbm_params)
  lgbm.fit(X_train,y_train)

  score = accuracy_score(y_test, lgbm.predict(X_test))

  return score

In [ ]:
lgbm_parameter_bounds = {
                      'learning_rate' : (0.01,0.5),
                      'n_estimators' : (50,250)
                      }

In [ ]:
BO_lgbm1 = BayesianOptimization(f = lgbm_bo1, pbounds = lgbm_parameter_bounds,random_state = 2023)

In [ ]:
BO_lgbm1.maximize(init_points = 15, n_iter = 5)

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.4263    | 0.1678    | 228.1     |
| 2         | 0.4263    | 0.2981    | 75.32     |
| 3         | 0.4263    | 0.07926   | 143.6     |
| 4         | 0.4263    | 0.02082   | 195.5     |
| 5         | 0.4263    | 0.2669    | 159.0     |
| 6         | 0.4263    | 0.2336    | 150.3     |
| 7         | 0.4263    | 0.2033    | 80.23     |
| 8         | 0.4263    | 0.1868    | 82.42     |
| 9         | 0.4263    | 0.1756    | 86.06     |
| 10        | 0.4263    | 0.2016    | 57.13     |
| 11        | 0.4263    | 0.2868    | 90.69     |
| 12        | 0.4263    | 0.1671    | 125.3     |
| 13        | 0.4263    | 0.1002    | 70.79     |
| 14        | 0.4263    | 0.2329    | 89.17     |
| 15        | 0.4263    | 0.1955    | 236.1     |
| 16        | 0.4263    | 0.3886    | 179.0     |
| 17        | 0.4263    | 0.325     | 250.0     |
| 18        | 0.4263    | 0.01905   | 250.0     |


In [ ]:
BO_lgbm1.max

{'target': 0.42629716981132076,
 'params': {'learning_rate': 0.1677742689560642,
  'n_estimators': 228.08449035789616}}

In [ ]:
lgbm_clf = GradientBoostingClassifier(random_state=2023,learning_rate=0.17,
                                n_estimators=228)
lgbm_clf.fit(X_train, y_train["수질등급"])
accuracy_train = accuracy_score(y_train["수질등급"], lgbm_clf.predict(X_train))
accuracy_test = accuracy_score(y_test["수질등급"], lgbm_clf.predict(X_test))
print('LGBM 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_train["수질등급"], lgbm_clf.predict(X_train)),'\n')
print('LGBM 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_test["수질등급"], lgbm_clf.predict(X_test)))

LGBM 학습 정확도: 0.6416
[[  77  459  289    0]
 [  38 4544 3801    1]
 [  25 1603 6606    0]
 [   0    7   56   14]] 

LGBM 정확도: 0.7252
[[   0   26    0    0]
 [   7 1464  698    0]
 [  17  615 2225    1]
 [   0    2   32    1]]


###5-1. XGB - 0.6223

In [ ]:
# train/test dataset 생성

train=tan2.loc[:,["날짜","수온","강우량","수질등급"]]
test=test2.loc[:,["날짜","수온","강우량","수질등급"]]

In [ ]:
X_train=train[['수온','강우량']]
y_train=train[['수질등급']]

X_test=test[['수온','강우량']]
y_test=test[['수질등급']]

In [ ]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [ ]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [ ]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [ ]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def xgb_bo1(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [ ]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [ ]:
BO_xgb1 = BayesianOptimization(f = xgb_bo1, pbounds = xgb_parameter_bounds,random_state = 2023)

In [ ]:
BO_xgb1.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.7199    | 0.3288    | 6.562     | 154.7     |
| 2         | 0.741     | 0.1353    | 3.565     | 141.5     |
| 3         | 0.7362    | 0.03187   | 5.909     | 147.7     |
| 4         | 0.7131    | 0.5495    | 4.825     | 145.2     |
| 5         | 0.7152    | 0.4005    | 3.605     | 129.7     |
| 6         | 0.7415    | 0.1705    | 4.352     | 109.8     |
| 7         | 0.7437    | 0.3971    | 3.143     | 152.1     |
| 8         | 0.738     | 0.2114    | 4.282     | 131.4     |
| 9         | 0.7408    | 0.1922    | 3.416     | 140.0     |
| 10        | 0.7154    | 0.2039    | 4.514     | 192.4     |
| 11        | 0.704     | 0.7626    | 6.083     | 155.6     |
| 12        | 0.7056    | 0.7937    | 6.241     | 197.9     |
| 13        | 0.7129    | 0.8859    | 3.439     | 180.2     |
| 14        | 0.717     | 0.3145    | 4.046     | 134.6     |
| 15    

In [ ]:
BO_xgb1.max

{'target': 0.7437106918238994,
 'params': {'learning_rate': 0.39708148712852204,
  'max_depth': 3.1425928416163527,
  'n_estimators': 152.13478138103207}}

In [ ]:
xgb_clf = XGBClassifier(n_estimators=131, learning_rate=0.12,
                        max_depth=5,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.6367
[[  73  456  296    0]
 [  38 4327 4019    0]
 [  25 1455 6754    0]
 [   0    2   74    1]] 

xgboost 예측 정확도: 0.7388
[[   0   26    0    0]
 [   4 1444  721    0]
 [  17  526 2315    0]
 [   0    0   35    0]]


###5-2 - 날짜 제외 모든 변수 - 0.8456

In [ ]:
# train/test dataset 생성

train=tan2.drop(columns=['날짜','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test2.drop(columns=['날짜','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [ ]:
X_train=train.drop('수질등급', axis=1)
y_train=train[['수질등급']]

X_test=test.drop('수질등급', axis=1)
y_test=test[['수질등급']]

In [ ]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [ ]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [ ]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [ ]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def xgb_bo2(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [ ]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [ ]:
BO_xgb2 = BayesianOptimization(f = xgb_bo2, pbounds = xgb_parameter_bounds,random_state = 2023)

In [ ]:
BO_xgb2.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.7864    | 0.3288    | 6.562     | 154.7     |
| 2         | 0.7893    | 0.1353    | 3.565     | 141.5     |
| 3         | 0.7934    | 0.03187   | 5.909     | 147.7     |
| 4         | 0.7917    | 0.5495    | 4.825     | 145.2     |
| 5         | 0.7838    | 0.4005    | 3.605     | 129.7     |
| 6         | 0.7862    | 0.1705    | 4.352     | 109.8     |
| 7         | 0.7911    | 0.3971    | 3.143     | 152.1     |
| 8         | 0.7877    | 0.2114    | 4.282     | 131.4     |
| 9         | 0.7901    | 0.1922    | 3.416     | 140.0     |
| 10        | 0.7877    | 0.2039    | 4.514     | 192.4     |
| 11        | 0.7773    | 0.7626    | 6.083     | 155.6     |
| 12        | 0.7852    | 0.7937    | 6.241     | 197.9     |
| 13        | 0.7773    | 0.8859    | 3.439     | 180.2     |
| 14        | 0.7911    | 0.3145    | 4.046     | 134.6     |
| 15    

In [ ]:
BO_xgb2.max

{'target': 0.7957940251572327,
 'params': {'learning_rate': 0.043263515100744686,
  'max_depth': 5.912930668731218,
  'n_estimators': 147.77878535401612}}

In [ ]:
xgb_clf = XGBClassifier(n_estimators=148, learning_rate=0.043,
                        max_depth=6,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.9312
[[ 659  166    0    0]
 [  64 7869  451    0]
 [   0  479 7754    1]
 [   0    0   44   33]] 

xgboost 예측 정확도: 0.7928
[[   0   26    0    0]
 [   0 1365  804    0]
 [   0  189 2669    0]
 [   0    0   35    0]]


###5-3 - 날짜, 연, 월, 일, 시 제외 - 0.8452

In [ ]:
tan2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   날짜         17520 non-null  object 
 1   연          17520 non-null  int64  
 2   월          17520 non-null  int64  
 3   일          17520 non-null  int64  
 4   시          17520 non-null  int64  
 5   수온         17520 non-null  float64
 6   pH         17520 non-null  float64
 7   용존산소(㎎/L)  17520 non-null  float64
 8   총질소(㎎/L)   17520 non-null  float64
 9   총인(㎎/L)    17520 non-null  float64
 10  총유기탄소      17520 non-null  float64
 11  강우량        17520 non-null  float64
 12  수질등급       17520 non-null  int64  
 13  수질등급2      17520 non-null  int64  
 14  pH점수       17520 non-null  int64  
 15  용존산소점수     17520 non-null  int64  
 16  총인점수       17520 non-null  int64  
 17  총유기탄소점수    17520 non-null  int64  
dtypes: float64(7), int64(10), object(1)
memory usage: 2.4+ MB


In [ ]:
# train/test dataset 생성

train=tan2.drop(columns=['날짜','연','월','일','시','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test2.drop(columns=['날짜','연','월','일','시','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [ ]:
X_train=train.drop('수질등급', axis=1)
y_train=train[['수질등급']]

X_test=test.drop('수질등급', axis=1)
y_test=test[['수질등급']]

In [ ]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [ ]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [ ]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [ ]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def xgb_bo3(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [ ]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [ ]:
BO_xgb3 = BayesianOptimization(f = xgb_bo3, pbounds = xgb_parameter_bounds,random_state = 2023)

In [ ]:
BO_xgb3.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.7691    | 0.3288    | 6.562     | 154.7     |
| 2         | 0.7693    | 0.1353    | 3.565     | 141.5     |
| 3         | 0.7697    | 0.03187   | 5.909     | 147.7     |
| 4         | 0.7691    | 0.5495    | 4.825     | 145.2     |
| 5         | 0.7689    | 0.4005    | 3.605     | 129.7     |
| 6         | 0.7697    | 0.1705    | 4.352     | 109.8     |
| 7         | 0.7691    | 0.3971    | 3.143     | 152.1     |
| 8         | 0.7691    | 0.2114    | 4.282     | 131.4     |
| 9         | 0.7702    | 0.1922    | 3.416     | 140.0     |
| 10        | 0.7689    | 0.2039    | 4.514     | 192.4     |
| 11        | 0.7691    | 0.7626    | 6.083     | 155.6     |
| 12        | 0.7691    | 0.7937    | 6.241     | 197.9     |
| 13        | 0.7691    | 0.8859    | 3.439     | 180.2     |
| 14        | 0.7689    | 0.3145    | 4.046     | 134.6     |
| 15    

In [ ]:
BO_xgb3.max

{'target': 0.7702437106918238,
 'params': {'learning_rate': 0.19221360040929075,
  'max_depth': 3.415807345411713,
  'n_estimators': 140.04199470019825}}

In [ ]:
xgb_clf = XGBClassifier(n_estimators=148, learning_rate=0.032,
                        max_depth=6,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 1.0000
[[ 825    0    0    0]
 [   0 8384    0    0]
 [   0    0 8234    0]
 [   0    0    0   77]] 

xgboost 예측 정확도: 0.7697
[[  26    0    0    0]
 [  37 2106   26    0]
 [   0 1092 1766    0]
 [   0    0   17   18]]


# **파생변수 만들기**

### 1. 과거 강우량 변수

In [215]:
tan2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   날짜         17520 non-null  datetime64[ns]
 1   연          17520 non-null  int64         
 2   월          17520 non-null  int64         
 3   일          17520 non-null  int64         
 4   시          17520 non-null  int64         
 5   수온         17520 non-null  float64       
 6   pH         17520 non-null  float64       
 7   용존산소(㎎/L)  17520 non-null  float64       
 8   총질소(㎎/L)   17520 non-null  float64       
 9   총인(㎎/L)    17520 non-null  float64       
 10  총유기탄소      17520 non-null  float64       
 11  강우량        17520 non-null  float64       
 12  수질등급       17520 non-null  int64         
 13  수질등급2      17520 non-null  int64         
 14  pH점수       17520 non-null  int64         
 15  용존산소점수     17520 non-null  int64         
 16  총인점수       17520 non-null  int64        

In [216]:
tan2['날짜']=pd.to_datetime(tan2['날짜'], format='%Y/%m/%d %H:%M')

In [217]:
from datetime import timedelta

hour4 = tan2[['날짜','강우량']]
hour4['날짜'] = hour4['날짜'] + timedelta(hours=4)
hour4.columns=['날짜','4시간 전 강우량']

hour8 = tan2[['날짜','강우량']]
hour8['날짜'] = hour8['날짜'] + timedelta(hours=8)
hour8.columns=['날짜','8시간 전 강우량']

hour16 = tan2[['날짜','강우량']]
hour16['날짜'] = hour16['날짜'] + timedelta(hours=16)
hour16.columns=['날짜','16시간 전 강우량']

day1 = tan2[['날짜','강우량']]
day1['날짜'] = day1['날짜'] + timedelta(days=1)
day1.columns=['날짜','1일 전 강우량']

day3 = tan2[['날짜','강우량']]
day3['날짜'] = day3['날짜'] + timedelta(days=3)
day3.columns=['날짜','3일 전 강우량']

day5 = tan2[['날짜','강우량']]
day5['날짜'] = day5['날짜'] + timedelta(days=5)
day5.columns=['날짜','5일 전 강우량']

day7 = tan2[['날짜','강우량']]
day7['날짜'] = day7['날짜'] + timedelta(days=7)
day7.columns=['날짜','7일 전 강우량']

week2 = tan2[['날짜','강우량']]
week2['날짜'] = week2['날짜'] + timedelta(weeks=2)
week2.columns=['날짜','2주 전 강우량']

week3 = tan2[['날짜','강우량']]
week3['날짜'] = week3['날짜'] + timedelta(weeks=3)
week3.columns=['날짜','3주 전 강우량']

week4 = tan2[['날짜','강우량']]
week4['날짜'] = week4['날짜'] + timedelta(weeks=4)
week4.columns=['날짜','4주 전 강우량']

In [218]:
from functools import reduce

# dfs = [tan2, hour8, hour16, day1, day3, day5, day7, week2, week3, week4]
dfs = [tan2,hour4, hour8, hour16, day1, day3, day5, day7, week2]
df_merge = reduce(lambda left, right: pd.merge(left, right, how='left',on='날짜'), dfs)

In [219]:
tan3 = df_merge.loc[336:,:]

tan3 = tan3[['날짜','연','월','일','시','수온','강우량','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총질소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수','수질등급']]

tan3.drop('총질소(㎎/L)',axis=1,inplace=True)

In [220]:
# test 데이터도 똑같이 전처리

# 과거 강우량 데이터 생성
test2['날짜']=pd.to_datetime(test2['날짜'], format='%Y/%m/%d %H:%M')

hour4 = test2[['날짜','강우량']]
hour4['날짜'] = hour4['날짜'] + timedelta(hours=4)
hour4.columns=['날짜','4시간 전 강우량']

hour8 = test2[['날짜','강우량']]
hour8['날짜'] = hour8['날짜'] + timedelta(hours=8)
hour8.columns=['날짜','8시간 전 강우량']

hour16 = test2[['날짜','강우량']]
hour16['날짜'] = hour16['날짜'] + timedelta(hours=16)
hour16.columns=['날짜','16시간 전 강우량']

day1 = test2[['날짜','강우량']]
day1['날짜'] = day1['날짜'] + timedelta(days=1)
day1.columns=['날짜','1일 전 강우량']

day3 = test2[['날짜','강우량']]
day3['날짜'] = day3['날짜'] + timedelta(days=3)
day3.columns=['날짜','3일 전 강우량']

day5 = test2[['날짜','강우량']]
day5['날짜'] = day5['날짜'] + timedelta(days=5)
day5.columns=['날짜','5일 전 강우량']

day7 = test2[['날짜','강우량']]
day7['날짜'] = day7['날짜'] + timedelta(days=7)
day7.columns=['날짜','7일 전 강우량']

week2 = test2[['날짜','강우량']]
week2['날짜'] = week2['날짜'] + timedelta(weeks=2)
week2.columns=['날짜','2주 전 강우량']

# 데이터프레임 병합
dfs = [test2,hour4, hour8, hour16, day1, day3, day5, day7, week2]
test_merge = reduce(lambda left, right: pd.merge(left, right, how='left',on='날짜'), dfs)

# 결측일자 제거
test3 = test_merge.loc[336:,:]

# 칼럼 재정렬
test3 = test3[['날짜','연','월','일','시','수온','강우량','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총질소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수','수질등급']]
# 불필요칼럼 제거
test3.drop('총질소(㎎/L)',axis=1,inplace=True)

# 확인
test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4752 entries, 336 to 5087
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          4752 non-null   datetime64[ns]
 1   연           4752 non-null   int64         
 2   월           4752 non-null   int64         
 3   일           4752 non-null   int64         
 4   시           4752 non-null   int64         
 5   수온          4752 non-null   float64       
 6   강우량         4752 non-null   float64       
 7   4시간 전 강우량   4752 non-null   float64       
 8   8시간 전 강우량   4752 non-null   float64       
 9   16시간 전 강우량  4752 non-null   float64       
 10  1일 전 강우량    4752 non-null   float64       
 11  3일 전 강우량    4752 non-null   float64       
 12  5일 전 강우량    4752 non-null   float64       
 13  7일 전 강우량    4752 non-null   float64       
 14  2주 전 강우량    4752 non-null   float64       
 15  pH          4752 non-null   float64       
 16  용존산소(㎎/L)   4752 non-n

### 2. 범주형 변수

In [221]:
def season(x):
  if ((x>=3) & (x<=5)):
    return 'spring'
  elif ((x>=6) & (x<=8)):
    return 'summer'
  elif ((x>=9) & (x<=11)):
    return 'fall'
  else:
    return 'winter'

tan3['계절'] = tan3['월'].apply(season)
test3['계절'] = test3['월'].apply(season)

In [222]:
tan3['홍수기']=0
test3['홍수기']=0

for i in range(tan3.shape[0]):
  M=tan3.iloc[i]["월"]
  D=tan3.iloc[i]["일"]
  if M==7 or M==8:
    tan3.iloc[i]["홍수기"]="홍수기"
  elif M==6 and D>=21:
    tan3.iloc[i]["홍수기"]="홍수기"
  elif M==9 and D<21:
    tan3.iloc[i]["홍수기"]="홍수기"
  else:
    tan3.iloc[i]["홍수기"]="갈수기"

for i in range(test3.shape[0]):
  M=test3.iloc[i]["월"]
  D=test3.iloc[i]["일"]
  if M==7 or M==8:
    test3.iloc[i]["홍수기"]="홍수기"
  elif M==6 and D>=21:
    test3.iloc[i]["홍수기"]="홍수기"
  elif M==9 and D<21:
    test3.iloc[i]["홍수기"]="홍수기"
  else:
    test3.iloc[i]["홍수기"]="갈수기"

# **다시 모델링 - 4개 등급으로**

In [223]:
tan3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17184 entries, 336 to 17519
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          17184 non-null  datetime64[ns]
 1   연           17184 non-null  int64         
 2   월           17184 non-null  int64         
 3   일           17184 non-null  int64         
 4   시           17184 non-null  int64         
 5   수온          17184 non-null  float64       
 6   강우량         17184 non-null  float64       
 7   4시간 전 강우량   17184 non-null  float64       
 8   8시간 전 강우량   17184 non-null  float64       
 9   16시간 전 강우량  17184 non-null  float64       
 10  1일 전 강우량    17184 non-null  float64       
 11  3일 전 강우량    17184 non-null  float64       
 12  5일 전 강우량    17184 non-null  float64       
 13  7일 전 강우량    17184 non-null  float64       
 14  2주 전 강우량    17184 non-null  float64       
 15  pH          17184 non-null  float64       
 16  용존산소(㎎/L)   17184 no

In [224]:
#함수적용 _ train
get_pH(tan3)
get_do(tan3)
get_tp(tan3)
get_toc(tan3)
get_grade25(tan3)

#함수적용 _ test
get_pH(test3)
get_do(test3)
get_tp(test3)
get_toc(test3)
get_grade25(test3)

In [225]:
# train/test dataset 생성

# train=tan3.drop(columns=['날짜','계절','연','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
# test=test3.drop(columns=['날짜','계절','연','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
train=tan3.drop(columns=['날짜','계절','홍수기','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test3.drop(columns=['날짜','계절','홍수기','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [226]:
X_train=train.drop('수질등급', axis=1)
y_train=train[['수질등급']]

X_test=test.drop('수질등급', axis=1)
y_test=test[['수질등급']]

In [227]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [228]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [229]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [230]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [231]:
def xgb_bo2(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [232]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [233]:
BO_xgb2 = BayesianOptimization(f = xgb_bo2, pbounds = xgb_parameter_bounds,random_state = 2023)

In [234]:
BO_xgb2.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.6898    | 0.3288    | 6.562     | 154.7     |
| 2         | 0.7256    | 0.1353    | 3.565     | 141.5     |
| 3         | 0.7229    | 0.03187   | 5.909     | 147.7     |
| 4         | 0.6913    | 0.5495    | 4.825     | 145.2     |
| 5         | 0.6959    | 0.4005    | 3.605     | 129.7     |
| 6         | 0.7252    | 0.1705    | 4.352     | 109.8     |
| 7         | 0.7052    | 0.3971    | 3.143     | 152.1     |
| 8         | 0.7155    | 0.2114    | 4.282     | 131.4     |
| 9         | 0.7256    | 0.1922    | 3.416     | 140.0     |
| 10        | 0.6949    | 0.2039    | 4.514     | 192.4     |
| 11        | 0.6738    | 0.7626    | 6.083     | 155.6     |
| 12        | 0.6782    | 0.7937    | 6.241     | 197.9     |
| 13        | 0.6858    | 0.8859    | 3.439     | 180.2     |
| 14        | 0.7018    | 0.3145    | 4.046     | 134.6     |
| 15    

In [235]:
BO_xgb2.max

{'target': 0.735479797979798,
 'params': {'learning_rate': 0.01,
  'max_depth': 4.808306812339451,
  'n_estimators': 109.1180224549561}}

In [236]:
xgb_clf = XGBClassifier(n_estimators=142, learning_rate=0.2207,
                        max_depth=3,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.8536
[[ 426  404    0    0]
 [  92 6896 1083    0]
 [   1  892 7309    3]
 [   0    2   39   37]] 

xgboost 예측 정확도: 0.7258
[[   0   15    0    0]
 [   0  999 1007    0]
 [   0  244 2448    5]
 [   0    0   32    2]]


0. 몇시간전 강우량 없는거 -> 0.7295
1. 날짜만 제외 -> 0.7298
2. 날짜,연 제외 -> 0.7336
3. 날짜,연,2주전 제외 -> 0.7355

- 17520개 있는거는 날짜만 없는게 0.7928정도였는데 이거 행이 몇개 없어진것 때문에 그런듯,
    - 데이터 긁어올때 과거 데이터 2주치 더 긁어와서 대입 후에 하면 성능 더 오를지도

In [238]:
tan2.head()

,날짜,연,월,일,시,수온,pH,용존산소(㎎/L),총질소(㎎/L),총인(㎎/L),총유기탄소,강우량,수질등급,수질등급2,pH점수,용존산소점수,총인점수,총유기탄소점수
0,2021-01-01 00:00:00,2021,1,1,0,7.0,7.2,10.0,10.34,0.080,8.1,0.0,2,6,3,4,3,1
1,2021-01-01 01:00:00,2021,1,1,1,7.0,7.2,10.0,10.34,0.080,8.1,0.0,2,6,3,4,3,1
2,2021-01-01 02:00:00,2021,1,1,2,6.9,7.2,10.0,10.32,0.085,7.1,0.0,2,5,3,4,3,1
3,2021-01-01 03:00:00,2021,1,1,3,6.9,7.2,9.9,10.16,0.100,7.4,0.0,2,5,3,4,3,1
4,2021-01-01 04:00:00,2021,1,1,4,6.8,7.2,9.9,10.32,0.095,8.0,0.0,2,5,3,4,3,1


# **2020년 12월 데이터 보충**

### 일단 12월 데이터 다시 전처리

In [239]:
water20 = pd.read_csv('2020수질.csv', encoding='cp949')
rain20 = pd.read_csv('2020강우량.csv', encoding='cp949')

In [240]:
rain20 = rain20[rain20['구청명']=='송파구']

In [241]:
# 수질 칼럼명 적용
water20.columns = water20.iloc[0]
water20.drop(0,axis=0,inplace=True)

# 페놀, 시안 칼럼 제거(모든 값 0)
water20.drop(['페놀','시안'], axis=1, inplace=True)

water20.head()

,날짜,시간,측정소명,수온,pH,용존산소(㎎/L),총질소(㎎/L),총인(㎎/L),총유기탄소
1,20210101,24:00:00,탄천_자동,8.4,7.2,9.7,10.48,0.085,8
2,20210101,23:00,탄천_자동,9,7.1,9.6,10.76,0.085,7.4
3,20210101,22:00,탄천_자동,9.3,7.1,9.4,10.44,0.08,8.4
4,20210101,21:00,탄천_자동,9.4,7.2,9.4,10.18,0.08,7.6
5,20210101,20:00,탄천_자동,9.5,7.2,9.6,9.96,0.08,7.1


In [242]:
# 수질 데이터 datetime 형변환
water20.loc[water20["시간"]=="24:00:00","시간"]="00:00"

water20['Date'] = water20["날짜"].map(str) + " " + water20["시간"] # 년/월/일/시/분 통합

water20['Date']=pd.to_datetime(water20['Date'], format='%Y%m%d %H:%M')
water20['날짜']=pd.to_datetime(water20['날짜'], format='%Y%m%d')

water20.loc[water20["시간"]=="00:00","날짜"]=water20.loc[water20["시간"]=="00:00","날짜"]+pd.Timedelta(days=1)
water20.loc[water20["시간"]=="00:00","Date"]=water20.loc[water20["시간"]=="00:00","Date"]+pd.Timedelta(days=1)

# 연,월,일,시,분 칼럼 생성
water20['year'] = water20['Date'].dt.year
water20['month'] = water20['Date'].dt.month
water20['day'] = water20['Date'].dt.day
water20['hour'] = water20['Date'].dt.hour
water20['minute'] = water20['Date'].dt.minute

In [243]:
# 강우량 데이터 datetime 형변환
rain20['자료수집 시각']=pd.to_datetime(rain20['자료수집 시각'])

# 연,월,일,시,분 칼럼 생성
rain20['year'] = rain20['자료수집 시각'].dt.year # 년도
rain20['month'] = rain20['자료수집 시각'].dt.month # 월
rain20['day'] = rain20['자료수집 시각'].dt.day     # 일
rain20['hour'] = rain20['자료수집 시각'].dt.hour   # 시
rain20['minute'] = rain20['자료수집 시각'].dt.minute   # 분

In [244]:
# 이전 값으로 결측치 대체

from scipy import interpolate
water20 = water20.interpolate(method='pad')

In [245]:
rain20.isna().sum()

강우량계 코드    0
강우량계명      0
구청 코드      0
구청명        0
10분우량      0
자료수집 시각    0
year       0
month      0
day        0
hour       0
minute     0
dtype: int64

In [246]:
water20.isna().sum()

0
날짜           0
시간           0
측정소명         0
수온           0
pH           0
용존산소(㎎/L)    0
총질소(㎎/L)     0
총인(㎎/L)      0
총유기탄소        0
Date         0
year         0
month        0
day          0
hour         0
minute       0
dtype: int64

In [247]:
# 2020-12-18 00:00 ~ 2020-12-31 23:00 데이터 프레임 생성

calendar = pd.date_range(start='2020-12-18',end='2021-1-1',freq='H')
cal20 = pd.DataFrame({'날짜':calendar})
cal20.drop(336,axis=0,inplace=True) #잘못생성된 2023년 데이터 제거

# 연, 월, 일, 시 칼럼 생성
cal20['연'] = cal20['날짜'].dt.year
cal20['월'] = cal20['날짜'].dt.month
cal20['일'] = cal20['날짜'].dt.day
cal20['시'] = cal20['날짜'].dt.hour

In [248]:
cal20.shape # 총 336개 행인지 확인

(336, 5)

In [249]:
# 불필요한 칼럼 제거
water20.drop(['날짜','시간','측정소명'],axis=1,inplace=True)

# 날짜 기준으로 데이터 병합
df = pd.merge(cal20,water20,how='left',left_on='날짜',right_on='Date')

# 중복되는 칼럼 제거
df.drop(['Date','year','month','day','hour','minute'],axis=1,inplace=True)

In [250]:
# 강우량계에 따라 데이터프레임 분리
rain_1_20 = rain20[rain20['강우량계명']=='마천2동']
rain_2_20 = rain20[rain20['강우량계명']=='송파구청']

In [251]:
# 10분마다 측정된 강우량을 시간당 누적 강우량 값으로 변경

# 마천2동
rain_1_20 = rain_1_20.groupby(['year','month','day','hour']).agg({'10분우량':'sum'}).reset_index()
rain_1_20.rename(columns={'10분우량':'강우량1'},inplace=True)

# 송파구
rain_2_20 = rain_2_20.groupby(['year','month','day','hour']).agg({'10분우량':'sum'}).reset_index()
rain_2_20.rename(columns={'10분우량':'강우량2'},inplace=True)

In [252]:
# 날짜 기준으로 데이터 병합을 위해 datetime형 칼럼 생성

# 문자열로 변환
rain_1_20['year'] = rain_1_20['year'].apply(str)
rain_1_20['month'] = rain_1_20['month'].apply(str)
rain_1_20['day'] = rain_1_20['day'].apply(str)
rain_1_20['hour'] = rain_1_20['hour'].apply(str)
# 월, 일 2자리수로 변환
rain_1_20['month'] = rain_1_20.apply(lambda x: x['month'].zfill(2),axis=1)
rain_1_20['day'] = rain_1_20.apply(lambda x: x['day'].zfill(2),axis=1)
# 날짜 형태로 문자열 재배치 후 형변환
rain_1_20['date'] = rain_1_20['year']+rain_1_20['month']+rain_1_20['day']+' '+rain_1_20['hour']+':00'
rain_1_20['date'] = pd.to_datetime(rain_1_20['date'], format='%Y%m%d %H:%M')

# 위와 동일한 과정 적용
rain_2_20['year'] = rain_2_20['year'].apply(str)
rain_2_20['month'] = rain_2_20['month'].apply(str)
rain_2_20['day'] = rain_2_20['day'].apply(str)
rain_2_20['hour'] = rain_2_20['hour'].apply(str)

rain_2_20['month'] = rain_2_20.apply(lambda x: x['month'].zfill(2),axis=1)
rain_2_20['day'] = rain_2_20.apply(lambda x: x['day'].zfill(2),axis=1)

rain_2_20['date'] = rain_2_20['year']+rain_2_20['month']+rain_2_20['day']+' '+rain_2_20['hour']+':00'
rain_2_20['date'] = pd.to_datetime(rain_2_20['date'], format='%Y%m%d %H:%M')

In [253]:
# 앞서 처리한 수질 데이터와 날짜 기준으로 병합 - 강우량1
df1 = pd.merge(df,rain_1_20,how='left',left_on='날짜',right_on='date')

# 중복되는 칼럼 제거
df1.drop(['year','month','day','hour','date'],axis=1,inplace=True)

# 날짜 기준으로 데이터 병합 - 강우량2
df2 = pd.merge(df1,rain_2_20,how='left',left_on='날짜',right_on='date')

# 중복되는 칼럼 제거
df2.drop(['year','month','day','hour','date'],axis=1,inplace=True)

# 중복 행 제거
df2.drop_duplicates(inplace=True)

In [254]:
# 강우량 값 평균 계산 후 칼럼 생성

# 함수 생성
def rain_mean(x,y):
  if (pd.isna(x)==True) & (pd.isna(y)==True):
    return np.NaN
  elif (pd.isna(x)!=True) & (pd.isna(y)==True):
    return x
  elif (pd.isna(x)==True) & (pd.isna(y)!=True):
    return y
  else:
    return (x+y)/2

# 함수 적용
df2['강우량'] = df2.apply(lambda x: rain_mean(x['강우량1'],x['강우량2']),axis=1)

# 강우량1, 강우량2 칼럼 제거
df3 = df2.drop(['강우량1','강우량2'],axis=1)

In [255]:
# 강우량 제외한 수질 관련 데이터는 전값으로 대체
df3['수온'] = df3['수온'].interpolate(method='pad')
df3['pH'] = df3['pH'].interpolate(method='pad')
df3['용존산소(㎎/L)'] = df3['용존산소(㎎/L)'].interpolate(method='pad')
df3['총질소(㎎/L)'] = df3['총질소(㎎/L)'].interpolate(method='pad')
df3['총인(㎎/L)'] = df3['총인(㎎/L)'].interpolate(method='pad')
df3['총유기탄소'] = df3['총유기탄소'].interpolate(method='pad')

# 제일 첫 데이터는 이전 값으로 대체 불가능 -> 이후 값으로 대체
df3['수온'] = df3['수온'].interpolate(method='bfill')
df3['pH'] = df3['pH'].interpolate(method='bfill')
df3['용존산소(㎎/L)'] = df3['용존산소(㎎/L)'].interpolate(method='bfill')
df3['총질소(㎎/L)'] = df3['총질소(㎎/L)'].interpolate(method='bfill')
df3['총인(㎎/L)'] = df3['총인(㎎/L)'].interpolate(method='bfill')
df3['총유기탄소'] = df3['총유기탄소'].interpolate(method='bfill')

In [256]:
df3.fillna(0, inplace=True)

In [257]:
tan20 = df3.copy()

In [258]:
test20 = tan2.loc[17184:17519,:]

In [259]:
test20 = test20.reset_index()
test20.drop(columns='index',axis=1,inplace=True)

### 합치기

In [260]:
tan20['수온'] = tan20['수온'].astype(float)
tan20['pH'] = tan20['pH'].astype(float)
tan20['용존산소(㎎/L)'] = tan20['용존산소(㎎/L)'].astype(float)
tan20['총질소(㎎/L)'] = tan20['총질소(㎎/L)'].astype(float)
tan20['총인(㎎/L)'] = tan20['총인(㎎/L)'].astype(float)
tan20['총유기탄소'] = tan20['총유기탄소'].astype(float)

In [261]:
# 함수 생성
def water_quality2(a,b,d,e):
  if (a>=6.5) & (a<=8.5) & (b>=7.5) & (d<=0.01) & (e<=2):
    return 1
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.02) & (e<=3):
    return 2
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.03) & (e<=4):
    return 3
  elif (a>=6.5) & (a<=8.5) & (b>=5) & (d<=0.05) & (e<=5):
    return 4
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (d<=0.1) & (e<=8):
    return 5
  elif (a>=6.0) & (a<=8.5) & (b>=2) & (d<=0.15) & (e<=10):
    return 6
  else:
    return 7

# 함수 적용
tan20['수질등급2'] = tan20.apply(lambda x: water_quality2(x['pH'],x['용존산소(㎎/L)'],x['총인(㎎/L)'],x['총유기탄소']), axis=1)

In [262]:
#함수적용
get_pH(tan20)
get_do(tan20)
get_tp(tan20)
get_toc(tan20)
get_grade25(tan20)

In [263]:
test20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   날짜         336 non-null    datetime64[ns]
 1   연          336 non-null    int64         
 2   월          336 non-null    int64         
 3   일          336 non-null    int64         
 4   시          336 non-null    int64         
 5   수온         336 non-null    float64       
 6   pH         336 non-null    float64       
 7   용존산소(㎎/L)  336 non-null    float64       
 8   총질소(㎎/L)   336 non-null    float64       
 9   총인(㎎/L)    336 non-null    float64       
 10  총유기탄소      336 non-null    float64       
 11  강우량        336 non-null    float64       
 12  수질등급       336 non-null    int64         
 13  수질등급2      336 non-null    int64         
 14  pH점수       336 non-null    int64         
 15  용존산소점수     336 non-null    int64         
 16  총인점수       336 non-null    int64         
 1

In [264]:
tan20 = tan20[['날짜','연','월','일','시','수온','pH','용존산소(㎎/L)','총질소(㎎/L)','총인(㎎/L)','총유기탄소','강우량','수질등급','수질등급2','pH점수','용존산소점수','총인점수','총유기탄소점수']]

In [265]:
tan20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   날짜         336 non-null    datetime64[ns]
 1   연          336 non-null    int64         
 2   월          336 non-null    int64         
 3   일          336 non-null    int64         
 4   시          336 non-null    int64         
 5   수온         336 non-null    float64       
 6   pH         336 non-null    float64       
 7   용존산소(㎎/L)  336 non-null    float64       
 8   총질소(㎎/L)   336 non-null    float64       
 9   총인(㎎/L)    336 non-null    float64       
 10  총유기탄소      336 non-null    float64       
 11  강우량        336 non-null    float64       
 12  수질등급       336 non-null    int64         
 13  수질등급2      336 non-null    int64         
 14  pH점수       336 non-null    int64         
 15  용존산소점수     336 non-null    int64         
 16  총인점수       336 non-null    int64         
 1

In [266]:
tan4 = pd.concat([tan20,tan2])
test4 = pd.concat([test20,test2])

In [267]:
tan4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17856 entries, 0 to 17519
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   날짜         17856 non-null  datetime64[ns]
 1   연          17856 non-null  int64         
 2   월          17856 non-null  int64         
 3   일          17856 non-null  int64         
 4   시          17856 non-null  int64         
 5   수온         17856 non-null  float64       
 6   pH         17856 non-null  float64       
 7   용존산소(㎎/L)  17856 non-null  float64       
 8   총질소(㎎/L)   17856 non-null  float64       
 9   총인(㎎/L)    17856 non-null  float64       
 10  총유기탄소      17856 non-null  float64       
 11  강우량        17856 non-null  float64       
 12  수질등급       17856 non-null  int64         
 13  수질등급2      17856 non-null  int64         
 14  pH점수       17856 non-null  int64         
 15  용존산소점수     17856 non-null  int64         
 16  총인점수       17856 non-null  int64        

In [268]:
test4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5424 entries, 0 to 5087
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   날짜         5424 non-null   datetime64[ns]
 1   연          5424 non-null   int64         
 2   월          5424 non-null   int64         
 3   일          5424 non-null   int64         
 4   시          5424 non-null   int64         
 5   수온         5424 non-null   float64       
 6   pH         5424 non-null   float64       
 7   용존산소(㎎/L)  5424 non-null   float64       
 8   총질소(㎎/L)   5424 non-null   float64       
 9   총인(㎎/L)    5424 non-null   float64       
 10  총유기탄소      5424 non-null   float64       
 11  강우량        5424 non-null   float64       
 12  수질등급       5424 non-null   int64         
 13  수질등급2      5424 non-null   int64         
 14  pH점수       5424 non-null   int64         
 15  용존산소점수     5424 non-null   int64         
 16  총인점수       5424 non-null   int64         


### 파생변수 생성

In [269]:
# 과거 강우량 데이터 생성
tan4['날짜']=pd.to_datetime(tan4['날짜'], format='%Y/%m/%d %H:%M')

hour4 = tan4[['날짜','강우량']]
hour4['날짜'] = hour4['날짜'] + timedelta(hours=4)
hour4.columns=['날짜','4시간 전 강우량']

hour8 = tan4[['날짜','강우량']]
hour8['날짜'] = hour8['날짜'] + timedelta(hours=8)
hour8.columns=['날짜','8시간 전 강우량']

hour16 = tan4[['날짜','강우량']]
hour16['날짜'] = hour16['날짜'] + timedelta(hours=16)
hour16.columns=['날짜','16시간 전 강우량']

day1 = tan4[['날짜','강우량']]
day1['날짜'] = day1['날짜'] + timedelta(days=1)
day1.columns=['날짜','1일 전 강우량']

day3 = tan4[['날짜','강우량']]
day3['날짜'] = day3['날짜'] + timedelta(days=3)
day3.columns=['날짜','3일 전 강우량']

day5 = tan4[['날짜','강우량']]
day5['날짜'] = day5['날짜'] + timedelta(days=5)
day5.columns=['날짜','5일 전 강우량']

day7 = tan4[['날짜','강우량']]
day7['날짜'] = day7['날짜'] + timedelta(days=7)
day7.columns=['날짜','7일 전 강우량']

week2 = tan4[['날짜','강우량']]
week2['날짜'] = week2['날짜'] + timedelta(weeks=2)
week2.columns=['날짜','2주 전 강우량']

# 데이터프레임 병합
dfs4 = [tan4,hour4, hour8, hour16, day1, day3, day5, day7, week2]
test4_merge = reduce(lambda left, right: pd.merge(left, right, how='left',on='날짜'), dfs4)

# 결측일자 제거
tan5 = test4_merge.loc[336:,:]

# 칼럼 재정렬
tan5 = tan5[['날짜','연','월','일','시','수온','강우량','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총질소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수','수질등급']]
# 불필요칼럼 제거
tan5.drop('총질소(㎎/L)',axis=1,inplace=True)

# 확인
tan5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 336 to 17855
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          17520 non-null  datetime64[ns]
 1   연           17520 non-null  int64         
 2   월           17520 non-null  int64         
 3   일           17520 non-null  int64         
 4   시           17520 non-null  int64         
 5   수온          17520 non-null  float64       
 6   강우량         17520 non-null  float64       
 7   4시간 전 강우량   17520 non-null  float64       
 8   8시간 전 강우량   17520 non-null  float64       
 9   16시간 전 강우량  17520 non-null  float64       
 10  1일 전 강우량    17520 non-null  float64       
 11  3일 전 강우량    17520 non-null  float64       
 12  5일 전 강우량    17520 non-null  float64       
 13  7일 전 강우량    17520 non-null  float64       
 14  2주 전 강우량    17520 non-null  float64       
 15  pH          17520 non-null  float64       
 16  용존산소(㎎/L)   17520 no

In [270]:
tan5['수질등급'].value_counts()

2    8384
3    8234
1     825
4      77
Name: 수질등급, dtype: int64

In [271]:
# 과거 강우량 데이터 생성
test4['날짜']=pd.to_datetime(test4['날짜'], format='%Y/%m/%d %H:%M')

hour4 = test4[['날짜','강우량']]
hour4['날짜'] = hour4['날짜'] + timedelta(hours=4)
hour4.columns=['날짜','4시간 전 강우량']

hour8 = test4[['날짜','강우량']]
hour8['날짜'] = hour8['날짜'] + timedelta(hours=8)
hour8.columns=['날짜','8시간 전 강우량']

hour16 = test4[['날짜','강우량']]
hour16['날짜'] = hour16['날짜'] + timedelta(hours=16)
hour16.columns=['날짜','16시간 전 강우량']

day1 = test4[['날짜','강우량']]
day1['날짜'] = day1['날짜'] + timedelta(days=1)
day1.columns=['날짜','1일 전 강우량']

day3 = test4[['날짜','강우량']]
day3['날짜'] = day3['날짜'] + timedelta(days=3)
day3.columns=['날짜','3일 전 강우량']

day5 = test4[['날짜','강우량']]
day5['날짜'] = day5['날짜'] + timedelta(days=5)
day5.columns=['날짜','5일 전 강우량']

day7 = test4[['날짜','강우량']]
day7['날짜'] = day7['날짜'] + timedelta(days=7)
day7.columns=['날짜','7일 전 강우량']

week2 = test4[['날짜','강우량']]
week2['날짜'] = week2['날짜'] + timedelta(weeks=2)
week2.columns=['날짜','2주 전 강우량']

# 데이터프레임 병합
dfs4 = [test4,hour4, hour8, hour16, day1, day3, day5, day7, week2]
test4_merge = reduce(lambda left, right: pd.merge(left, right, how='left',on='날짜'), dfs4)

# 결측일자 제거
test5 = test4_merge.loc[336:,:]

# 칼럼 재정렬
test5 = test5[['날짜','연','월','일','시','수온','강우량','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총질소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수','수질등급']]
# 불필요칼럼 제거
test5.drop('총질소(㎎/L)',axis=1,inplace=True)

# 확인
test5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5088 entries, 336 to 5423
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          5088 non-null   datetime64[ns]
 1   연           5088 non-null   int64         
 2   월           5088 non-null   int64         
 3   일           5088 non-null   int64         
 4   시           5088 non-null   int64         
 5   수온          5088 non-null   float64       
 6   강우량         5088 non-null   float64       
 7   4시간 전 강우량   5088 non-null   float64       
 8   8시간 전 강우량   5088 non-null   float64       
 9   16시간 전 강우량  5088 non-null   float64       
 10  1일 전 강우량    5088 non-null   float64       
 11  3일 전 강우량    5088 non-null   float64       
 12  5일 전 강우량    5088 non-null   float64       
 13  7일 전 강우량    5088 non-null   float64       
 14  2주 전 강우량    5088 non-null   float64       
 15  pH          5088 non-null   float64       
 16  용존산소(㎎/L)   5088 non-n

### 모델링

In [272]:
tan5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 336 to 17855
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          17520 non-null  datetime64[ns]
 1   연           17520 non-null  int64         
 2   월           17520 non-null  int64         
 3   일           17520 non-null  int64         
 4   시           17520 non-null  int64         
 5   수온          17520 non-null  float64       
 6   강우량         17520 non-null  float64       
 7   4시간 전 강우량   17520 non-null  float64       
 8   8시간 전 강우량   17520 non-null  float64       
 9   16시간 전 강우량  17520 non-null  float64       
 10  1일 전 강우량    17520 non-null  float64       
 11  3일 전 강우량    17520 non-null  float64       
 12  5일 전 강우량    17520 non-null  float64       
 13  7일 전 강우량    17520 non-null  float64       
 14  2주 전 강우량    17520 non-null  float64       
 15  pH          17520 non-null  float64       
 16  용존산소(㎎/L)   17520 no

In [273]:
# train/test dataset 생성

# train=tan5.drop(columns=['날짜','연','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
# test=test5.drop(columns=['날짜','연','4시간 전 강우량','8시간 전 강우량','16시간 전 강우량','1일 전 강우량','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
train=tan5.drop(columns=['날짜','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test5.drop(columns=['날짜','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [274]:
X_train=train.drop('수질등급', axis=1)
y_train=train[['수질등급']]

X_test=test.drop('수질등급', axis=1)
y_test=test[['수질등급']]

In [275]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [276]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [277]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [278]:
y_train.value_counts()

수질등급
2       8384
3       8234
1        825
4         77
dtype: int64

In [279]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [280]:
def xgb_bo2(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [281]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [282]:
BO_xgb2 = BayesianOptimization(f = xgb_bo2, pbounds = xgb_parameter_bounds,random_state = 2023)

In [286]:
BO_xgb2.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 26        | 0.7164    | 0.8571    | 6.159     | 99.66     |
| 27        | 0.7144    | 0.9337    | 5.0       | 130.1     |
| 28        | 0.7136    | 0.4888    | 5.061     | 199.4     |
| 29        | 0.717     | 0.7559    | 4.463     | 157.3     |
| 30        | 0.739     | 0.4254    | 3.367     | 148.8     |
| 31        | 0.7376    | 0.3474    | 3.073     | 156.4     |
| 32        | 0.7105    | 0.9226    | 4.125     | 181.8     |
| 33        | 0.7335    | 0.5541    | 3.087     | 128.2     |
| 34        | 0.7323    | 0.5545    | 3.053     | 168.7     |
| 35        | 0.7264    | 0.1979    | 5.97      | 159.2     |
| 36        | 0.7156    | 0.3686    | 5.508     | 146.6     |
| 37        | 0.7101    | 0.896     | 5.075     | 156.8     |
| 38        | 0.7189    | 0.4801    | 5.838     | 102.7     |
| 39        | 0.7042    | 0.7355    | 6.512     | 147.1     |
| 40    

In [287]:
BO_xgb2.max

{'target': 0.766312893081761,
 'params': {'learning_rate': 0.01,
  'max_depth': 4.808624380237392,
  'n_estimators': 109.11683135062572}}

In [288]:
xgb_clf = XGBClassifier(n_estimators=109, learning_rate=0.01,
                        max_depth=5,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.8056
[[  43  781    1    0]
 [   8 6846 1530    0]
 [   0 1019 7215    0]
 [   0    1   66   10]] 

xgboost 예측 정확도: 0.7663
[[   0   26    0    0]
 [   0 1229  940    0]
 [   0  188 2670    0]
 [   0    0   35    0]]


0. 파생변수(과거강우량),날짜 제외 -> 0.7295
1. 날짜만 제외 -> 0.7298
2. 날짜,연 제외 -> 0.7336
3. 날짜,연,2주전 제외 -> 0.7355
-------------------------------------------- 행 제거되는거 보충해서 17520개 채움
4. 날짜,연,2주전 제외 -> 0.7622(왜 다시하니까 0.7585 됐지)(또하니까 0.7663 됨)

- 17520개 있는거는 날짜만 없는게 0.7928정도였는데 이거 행이 몇개 없어진것 때문에 그런듯,
    - 데이터 긁어올때 과거 데이터 2주치 더 긁어와서 대입 후에 하면 성능 더 오를지도

### pH점수 예측 후 train에 포함해서 수질등급 예측

### pH점수 예측 모델링

In [289]:
tan5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 336 to 17855
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          17520 non-null  datetime64[ns]
 1   연           17520 non-null  int64         
 2   월           17520 non-null  int64         
 3   일           17520 non-null  int64         
 4   시           17520 non-null  int64         
 5   수온          17520 non-null  float64       
 6   강우량         17520 non-null  float64       
 7   4시간 전 강우량   17520 non-null  float64       
 8   8시간 전 강우량   17520 non-null  float64       
 9   16시간 전 강우량  17520 non-null  float64       
 10  1일 전 강우량    17520 non-null  float64       
 11  3일 전 강우량    17520 non-null  float64       
 12  5일 전 강우량    17520 non-null  float64       
 13  7일 전 강우량    17520 non-null  float64       
 14  2주 전 강우량    17520 non-null  float64       
 15  pH          17520 non-null  float64       
 16  용존산소(㎎/L)   17520 no

In [291]:
train=tan5.drop(columns=['날짜','연','월','일','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','수질등급','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test5.drop(columns=['날짜','연','월','일','3일 전 강우량','5일 전 강우량','7일 전 강우량','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','수질등급','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [292]:
X_train=train.drop('pH점수', axis=1)
y_train=train[['pH점수']]

X_test=test.drop('pH점수', axis=1)
y_test=test[['pH점수']]

In [293]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [294]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['pH점수']

  if a==1:
    y_tr_xgb.iloc[i]['pH점수']=0
  elif a==2:
    y_tr_xgb.iloc[i]['pH점수']=1
  elif a==3:
    y_tr_xgb.iloc[i]['pH점수']=2
  else:
    y_tr_xgb.iloc[i]['pH점수']=3

In [295]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['pH점수']

  if a==1:
    y_ts_xgb.iloc[i]['pH점수']=0
  elif a==2:
    y_ts_xgb.iloc[i]['pH점수']=1
  elif a==3:
    y_ts_xgb.iloc[i]['pH점수']=2
  else:
    y_ts_xgb.iloc[i]['pH점수']=3

In [296]:
def xgb_bo2(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [297]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [298]:
BO_xgb2 = BayesianOptimization(f = xgb_bo2, pbounds = xgb_parameter_bounds,random_state = 2023)

In [299]:
BO_xgb2.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.749     | 0.3288    | 6.562     | 154.7     |
| 2         | 0.775     | 0.1353    | 3.565     | 141.5     |
| 3         | 0.7893    | 0.03187   | 5.909     | 147.7     |
| 4         | 0.7514    | 0.5495    | 4.825     | 145.2     |
| 5         | 0.7583    | 0.4005    | 3.605     | 129.7     |
| 6         | 0.7726    | 0.1705    | 4.352     | 109.8     |
| 7         | 0.7642    | 0.3971    | 3.143     | 152.1     |
| 8         | 0.7573    | 0.2114    | 4.282     | 131.4     |
| 9         | 0.7761    | 0.1922    | 3.416     | 140.0     |
| 10        | 0.7567    | 0.2039    | 4.514     | 192.4     |
| 11        | 0.7439    | 0.7626    | 6.083     | 155.6     |
| 12        | 0.736     | 0.7937    | 6.241     | 197.9     |
| 13        | 0.7563    | 0.8859    | 3.439     | 180.2     |
| 14        | 0.762     | 0.3145    | 4.046     | 134.6     |
| 15    

In [300]:
BO_xgb2.max

{'target': 0.8125,
 'params': {'learning_rate': 0.01,
  'max_depth': 6.063046176976464,
  'n_estimators': 148.67551163923096}}

In [301]:
xgb_clf = XGBClassifier(n_estimators=149, learning_rate=0.01,
                        max_depth=6, random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.7654
[[   31   184     1     0]
 [    3 10783   653    20]
 [    0  2527  2466    44]
 [    0   364   314   130]] 

xgboost 예측 정확도: 0.8125
[[   4  147    3    0]
 [   1 3753  596    6]
 [   0  133  371   51]
 [   0    7   10    6]]


- pH점수 정확도 0.8125, pred를 학습자료로 사용

### 수질등급 예측 모델링

In [302]:
tan5['pH등급'] = xgb_clf.predict(X_train)
test5['pH등급'] = xgb_clf.predict(X_test)

In [303]:
train=tan5.drop(columns=['날짜','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)
test=test5.drop(columns=['날짜','연','2주 전 강우량','pH','용존산소(㎎/L)','총인(㎎/L)','총유기탄소','pH점수','용존산소점수','총인점수','총유기탄소점수'], axis=1)

In [304]:
X_train=train.drop('수질등급', axis=1)
y_train=train[['수질등급']]

X_test=test.drop('수질등급', axis=1)
y_test=test[['수질등급']]

In [305]:
y_tr_xgb=y_train.copy()
y_ts_xgb=y_test.copy()

In [306]:
for i in range(y_tr_xgb.shape[0]):
  a=y_tr_xgb.iloc[i]['수질등급']

  if a==1:
    y_tr_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_tr_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_tr_xgb.iloc[i]['수질등급']=2
  else:
    y_tr_xgb.iloc[i]['수질등급']=3

In [307]:
for i in range(y_ts_xgb.shape[0]):
  a=y_ts_xgb.iloc[i]['수질등급']

  if a==1:
    y_ts_xgb.iloc[i]['수질등급']=0
  elif a==2:
    y_ts_xgb.iloc[i]['수질등급']=1
  elif a==3:
    y_ts_xgb.iloc[i]['수질등급']=2
  else:
    y_ts_xgb.iloc[i]['수질등급']=3

In [308]:
def xgb_bo2(max_depth, learning_rate, n_estimators):
  xgb_params = {
      'max_depth' : int(round(max_depth)),
      'learning_rate' : learning_rate,
      'n_estimators' : int(round(n_estimators))
      }

  xgb = XGBClassifier(**xgb_params)
  xgb.fit(X_train,y_tr_xgb)

  score = accuracy_score(y_ts_xgb, xgb.predict(X_test))

  return score

In [309]:
xgb_parameter_bounds = {
                      'max_depth' : (3,7),
                      'learning_rate' : (0.01,1),
                      'n_estimators' : (90,200)
                      }

In [310]:
BO_xgb2 = BayesianOptimization(f = xgb_bo2, pbounds = xgb_parameter_bounds,random_state = 2023)

In [313]:
BO_xgb2.maximize(init_points = 20, n_iter = 5)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 26        | 0.7132    | 0.8571    | 6.159     | 99.66     |
| 27        | 0.7085    | 0.9337    | 5.0       | 130.1     |
| 28        | 0.7223    | 0.4888    | 5.061     | 199.4     |
| 29        | 0.7195    | 0.7559    | 4.463     | 157.3     |
| 30        | 0.7423    | 0.4254    | 3.367     | 148.8     |
| 31        | 0.7429    | 0.3474    | 3.073     | 156.4     |
| 32        | 0.7083    | 0.9226    | 4.125     | 181.8     |
| 33        | 0.7364    | 0.5541    | 3.087     | 128.2     |
| 34        | 0.7333    | 0.5545    | 3.053     | 168.7     |
| 35        | 0.7256    | 0.1979    | 5.97      | 159.2     |
| 36        | 0.7217    | 0.3686    | 5.508     | 146.6     |
| 37        | 0.7142    | 0.896     | 5.075     | 156.8     |
| 38        | 0.7223    | 0.4801    | 5.838     | 102.7     |
| 39        | 0.7099    | 0.7355    | 6.512     | 147.1     |
| 40    

In [314]:
BO_xgb2.max

{'target': 0.7686713836477987,
 'params': {'learning_rate': 0.07621641711792349,
  'max_depth': 3.3611934321753334,
  'n_estimators': 139.9896190669906}}

In [315]:
xgb_clf = XGBClassifier(n_estimators=140, learning_rate=0.0762,
                        max_depth=3,random_state = 2023)
xgb_clf.fit(X_train, y_tr_xgb)
accuracy_train = accuracy_score(y_tr_xgb, xgb_clf.predict(X_train))
accuracy_test = accuracy_score(y_ts_xgb, xgb_clf.predict(X_test))
print('xgboost 학습 정확도: {:.4f}'.format(accuracy_train))
print(confusion_matrix(y_tr_xgb, xgb_clf.predict(X_train)),'\n')
print('xgboost 예측 정확도: {:.4f}'.format(accuracy_test))
print(confusion_matrix(y_ts_xgb, xgb_clf.predict(X_test)))

xgboost 학습 정확도: 0.8186
[[ 197  627    1    0]
 [  80 6838 1466    0]
 [   1  936 7297    0]
 [   0    3   64   10]] 

xgboost 예측 정확도: 0.7695
[[   0   26    0    0]
 [   0 1233  936    0]
 [   0  176 2682    0]
 [   0    0   35    0]]
